# Estimating Fault Tolerant Reseources for Periodic Systems

The resource estimation code provided in this module relies on pyscf to compute the required symmetry adapted molecular orbitals, one- and two-electron integrals, and, correlated wavefunction calculations to determine various thresholds. So to start the tutorial let's run a periodic restriced Hartree--Fock (RHF) simulation of diamond. 

We assume a knowledge of electronic structure theory for solids, and in particular an understanding that any results need to be converged to the infintie $k$-point and complete basis set limit. For the purposes of this tutorial we will focus on results far away from this limit and simulate simple systems in minimal basis sets use very small $k$-point meshes. The module also assumes the use of density fitted integrals and we use range-separated density fitting throughout. 

In [3]:
from ase.build import bulk
import numpy as np

from pyscf.pbc import gto, scf
from pyscf.pbc.tools import pyscf_ase


# Build a 2 atom unit cell for carbon in the diamond structure near it's
# equilibrium lattice constant.
ase_atom = bulk("C", "diamond", a=3.5)
cell = gto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(ase_atom)
cell.a = ase_atom.cell[:].copy()
# Using a minimal basis set for expediency.
cell.basis = "gth-szv"
cell.pseudo = "gth-hf-rev"
cell.verbose = 0
cell.build()

# We are using a very small k-point mesh for speed purposes too.
kmesh = [1, 1, 3]
kpts = cell.make_kpts(kmesh)
num_kpts = len(kpts)
mf = scf.KRHF(cell, kpts).rs_density_fit() # Use range separated density fitting.
mf.kernel()
print("SCF energy: ", mf.e_tot)

# converged SCF energy with: -10.39193609748544

SCF energy:  -10.391936097489468


Armed with our SCF solution we can now generate the one and two-electron integrals required to compute $\lambda$.

In [7]:
# First build the one-body hamiltonian in the molecular orbital basis
hcore_mo = [C.conj().T @ hk @ C for (C, hk) in zip(mf.mo_coeff, mf.get_hcore())]
assert len(hcore_mo) == np.prod(kmesh)

In [13]:
# Get molecular orbital "Cholesky" integrals from RSGDF object, these are just
# 3-centre integrals (X|pq).
from kpoint_eri.factorizations.pyscf_chol_from_df import cholesky_from_df_ints  
chol = cholesky_from_df_ints(mf) 
print("(nkpts, nkpts) = {}".format(chol.shape))
print("(naux, nmo, nmo) = {}".format(chol[0,0].shape))

(nkpts, nkpts) = (3, 3)
(naux, nmo, nmo) = (108, 8, 8)


# Hamiltonian Representation
The resource estimation module provides four representations for the Hamiltonian: sparse, single-factorization, double-factorization, and tensor hypercontraction (THC). Each of these approaches introduces a different parameter which controls the accuracy of the factorization. Recall that we have

## Sparse Hamiltonian

$$

H = \sum_{pq\mathrm{k}} h_{pq}(\mathrm{k}) a

$$